Read only client paradata section from input text file.
Parse the client paradata by specific strings per pages/questions.

In [1]:
import csv
import datetime
today = datetime.datetime.now().date().strftime('%m%d%Y')
infile = open(r"L:\groups\TSG\dataops\Minako\Others\Emily\paradata\LS_CSP_test.txt", 'rb')
CSVoutfile = "L:\groups\TSG\dataops\Minako\Others\Emily\paradata\outpara_" + str(today) + ".csv"
outpara_sum = r"L:\groups\TSG\dataops\Minako\Others\Emily\paradata\outpara_summary" + str(today) + ".csv"
infile.next() # skip header

'"SID"^"csp"\r\n'

Read text file per line becuase This file has inconsistent delimited patterns.

In [2]:
# Function: read text file into dictionary
def csvtodict_para (csvFile):
    content = {}
    for line in infile:
        # It is ^ delimited, but ^ is used within client paradata as delimiter. Separate SID and client paradata.
        temp = line.split('^') 
        sid = temp[0] 
        paradata = line[len(sid)+1:]  # read lines except sid, which locates the beginning of line.
        content[sid[1:-1]] = {'sid':sid[1:-1],'data':str(paradata)[1:-1]} # remove quotechar by [1:-1]
    return content 

In [3]:
# String search function. It will find string between first and last.
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [4]:
# Fnction: export from dictionary to csv
def export_Dict2CSV (d_FPR, header, CSVoutfile):
    f = open(CSVoutfile, 'wb')
    cw = csv.writer(f, delimiter=',', quotechar='"')
    # Write headers #
    cw.writerow(header)
    # Write data to output file #
    for key in d_FPR.keys() :
        outdata = []
        for subkey in header:
            outdata.append(d_FPR[key][subkey])
        cw.writerow(outdata)

In [5]:
d = csvtodict_para(infile) # calling csvtodict_para function
print str(len(d)) + " rows in the input file"


7 rows in the input file


Setting up the default values.

In [6]:
minkey = None  # storage for minimum length of paradata among all data. This will be used to catch any questions missing.
maxkey = 0     # stodage for maximum length of paradata amound all data. This will be used to catch any questions missing.
out={}         # new dictionary to store paradata
count = 1      # count of paradata entries
checkpara = [] # storage to hold cases to check

Start loopin each sid in dictionary.

In [7]:
for key in d:
    text = d[key]['data'].split('^') # paradata is ^ delmited.
    if len(text) > maxkey: # updating maxkey
        maxkey = len(text)
    if minkey == None:     # updating minkey
        minkey = len(text)
    else:
        if minkey > len(text):
            minkey = len(text)
    # loop each items in paradata.
    for i in range(0,len(text)):
        # Split text with comma if it is not empty. 
        # The text for the first page, not item, has three items separated by comma.
        if text[i] != '':
            moretext = text[i].split(',')
            ##### page #####
            # time and answer is not available for page
            if 'onload' in text[i]: # page always has 'onload'.
                if len(moretext) == 3 : 
                    # position is set to start if onload- is in text
                    position = 'start' 
                    # if there is onload- in the text, question is between onload- and first =
                    question = find_between(moretext[0], 'onload-', '=') 
                    # timestamp is located at the 3rd item, after '='
                    timestamp = moretext[2].split('=')[-1]
                    out[count]= {'time':None, 'position':position, 'answer':None,'question':question, 
                                 'order':i, 'text':text[i], 'sid':key, 'timestamp':timestamp, 'type':'page'}
                    count= count + 1
                else:
                    # position is set to start if onload- is in text
                    position = 'start' 
                    # if there is onload- in the text, question is between onload- and first =
                    question = find_between(moretext[0], 'onload-', '=') 
                    if len(moretext) < 3:
                        timestamp = None
                    else:
                        timestamp = moretext[-1].split('=')[-1]
                    out[count]= {'time':None, 'position':position, 'answer':None,'question':question, 
                                 'order':i, 'text':text[i], 'sid':key, 'timestamp':timestamp, 'type':'page'}
                    count= count + 1                
            ##### item #####
            # timestamp is not available for item
            else:
                if len(moretext) == 1: 
                    # get the time, which is between t= and first :
                    time = find_between(text[i], 't=', ':') 
                    try :
                        temp = int(time)
                    except:
                        checkpara.append([key,moretext, len(text), text[i]])
                    # position is set to Noone for item
                    position = None
                    # if onload- is not in text, question is between first : and first =
                    question = find_between(text[i], ':', '=') 
                    # if onload- is not in text, answer is between the last = to the end
                    answer = text[i].split('=')[-1] 
                    out[count]= {'time':time, 'position':position, 'answer':answer,'question':question, 
                                 'order':i, 'text':text[i], 'sid':key, 'timestamp':None, 'type':'item'}
                    count= count + 1
                ##### item with (,) #####
                else:
                    for p in range(0,len(moretext)):
                        if 't=' == moretext[p][0:2]:
                            # get the time, which is between t= and first :
                            time = find_between(moretext[p], 't=', ':') 
                            try :
                                temp = int(time)
                            except:
                                checkpara.append([key,moretext[p], len(text), text[i]])
                            # position is set to Noone for item
                            position = None
                            # We need another logic to get the question, but for now, I just assume the question is :after
                            question = moretext[p].split(':')[-1]
                            # We need another logic to get the answer, for now, I just set it as None.
                            answer = None
                            out[count]= {'time':time, 'position':position, 'answer':answer,'question':question, 
                                         'order':i, 'text':text[i], 'sid':key, 'timestamp':None, 'type':'item'}
                            count= count + 1

In [8]:
print "keystroke - minimum length:" + str(minkey) + " maximum length:" + str(maxkey)
print str(len(checkpara)) + " SIDs have non-page and non-item paradata"
if len(checkpara) > 0:
    print checkpara[0] # this will print out the example of non-page non-item paradata SID.
print checkpara

keystroke - minimum length:1 maximum length:7763
1 SIDs have non-page and non-item paradata
['PT00002', ['ANS\r'], 1, 'ANS\r']
[['PT00002', ['ANS\r'], 1, 'ANS\r']]


In [9]:
outsummary={}
for count in out:
    # combination is sid and question. This will be the uniqueID for outsummary
    sidQ = out[count]['sid'] + "-" + out[count]['question'] 
    if sidQ not in outsummary:
        if out[count]['time'] not in (None, ""): # time doesn't exist all keystroke
            outsummary[sidQ] = {'sidQ':sidQ, 'sid':out[count]['sid'], 'question':out[count]['question'], 
                                'starttime':None, 'endtime':None,'maxorder':0,'count':1, 'answer':None, 
                                'sumtime':int(out[count]['time'])}
        else:
            outsummary[sidQ] = {'sidQ':sidQ, 'sid':out[count]['sid'], 'question':out[count]['question'], 
                                'starttime':None, 'endtime':None,'maxorder':0,'count':1, 'answer':None, 'sumtime':0}            
    else:
        outsummary[sidQ]['count'] = outsummary[sidQ]['count'] + 1 # count of question per sid
        if out[count]['time'] not in (None, ""): # time for page is None. Those were excluded.
            # sum time per question per sid
            outsummary[sidQ]['sumtime'] = outsummary[sidQ]['sumtime'] + int(out[count]['time']) 
    if out[count]['position'] == 'start': # position=start means keystroke includes onload-
        outsummary[sidQ]['starttime'] = out[count]['timestamp'] # if postion=start, starttime will be its time
        # maxorder will be updated to start order, if maxorder = 0
        if outsummary[sidQ]['maxorder'] == 0:
            outsummary[sidQ]['maxorder'] = out[count]['order'] 
    else:
        # if the keystroke order is bigger than the current maxorder
        if outsummary[sidQ]['maxorder'] < out[count]['order']: 
            outsummary[sidQ]['maxorder'] = out[count]['order'] # update maxorder
            outsummary[sidQ]['endtime'] = out[count]['time'] # update endtime
            outsummary[sidQ]['answer'] = out[count]['answer'] # update answer

Export client paradata and summary into csv

In [10]:
header = ['sid','type', 'order',  'question', 'time', 'answer', 'timestamp', 'position', 'text']
export_Dict2CSV (out, header, CSVoutfile)
header2 = ['sid',  'question', 'sidQ','maxorder','count', 'answer', 'sumtime']
export_Dict2CSV (outsummary, header2, outpara_sum)